<a href="https://www.kaggle.com/code/syedasimalishah/running-pandas-on-gpu?scriptVersionId=108885249" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cudf
import os
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

In [2]:
def rrt(dff):
    trainX = dff[['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x',
       'ball_vel_y', 'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z',
       'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y',
       'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x',
       'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost',
       'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z',
       'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y',
       'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x',
       'p5_vel_y', 'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer',
       'boost2_timer', 'boost3_timer', 'boost4_timer', 'boost5_timer']]
    trainYA = dff[['team_A_scoring_within_10sec']]
    trainYB = dff[['team_B_scoring_within_10sec']]
    
    f23 = sc.fit_transform(trainX.as_gpu_matrix())
    trainX = pd.DataFrame(f23)
    
    del dff,f23
    
    return trainX,trainYA,trainYB
def ttt(dff2):
    test = dff2[['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x',
       'ball_vel_y', 'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z',
       'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y',
       'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x',
       'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost',
       'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z',
       'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y',
       'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x',
       'p5_vel_y', 'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer',
       'boost2_timer', 'boost3_timer', 'boost4_timer', 'boost5_timer']]
    test.fillna(test.mean(), inplace=True)
    
    f24 = sc.fit_transform(test.as_gpu_matrix())
    test = pd.DataFrame(f24)
    
    del dff2,f24
    
    return test

In [3]:
#dtypes_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_dtypes.csv')
#dtypes = {k: v for (k, v) in zip(dtypes_df.column, dtypes_df.dtype)}
df = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_0.csv')
df.dropna(inplace=True)

In [4]:
trainX,trainYA,trainYB = rrt(df)

In [5]:
#f1 = cudf.concat([df,df1,df2,df3,df4])
#del df,df1,df2,df3,df4
#f2 = cudf.concat([df5,df6,df7,df8,df9])
#del df5,df6,df7,df8,df9
#df = cudf.concat([f1,f2])

In [6]:
dft = cudf.read_csv('/kaggle/input/tabular-playground-series-oct-2022/test.csv')

In [7]:
test = ttt(dft)

In [8]:
del df,dft

In [9]:
#from xgboost import XGBClassifier
#modelA = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#modelA.fit(trainXA,trainYA)
#modelB = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#modelB.fit(trainXB,trainYB)
#A = modelA.predict(test)
#B = modelB.predict(test)

In [10]:
#ss = pd.read_csv('../input/tabular-playground-series-oct-2022/sample_submission.csv')
#df = pd.DataFrame({'id':ss.id,'team_A_scoring_within_10sec':A,'team_B_scoring_within_10sec':B})
#df.to_csv('cudaf1.csv',index = False)
#modelA.save_model('modelA.json')
#modelB.save_model('modelB.json')
#trainYA=trainYA.to_pandas()

In [12]:
def mmm():
    from keras.models import Sequential
    from keras.layers import Dense
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.losses import BinaryCrossentropy
    model = Sequential()
    model.add(Dense(512, kernel_initializer='normal',activation='relu',input_shape=(54,)))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    return model

In [13]:
trainYA=trainYA.to_pandas()
trainYB=trainYB.to_pandas()

In [14]:
model1 = mmm()
model2 = mmm()

2022-10-23 07:55:26.977717: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 07:55:26.978621: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 07:55:26.992764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 07:55:26.993576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 07:55:26.994286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [16]:
model1.fit(trainX,trainYA,epochs=50,batch_size=10000)

2022-10-23 07:58:43.329107: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 673271568 exceeds 10% of free system memory.
2022-10-23 07:58:44.075679: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 673271568 exceeds 10% of free system memory.


Epoch 1/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0219 - accuracy: 0.9938
Epoch 2/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0196 - accuracy: 0.9945
Epoch 3/50
156/156 [==============================] - 2s 12ms/step - loss: 0.0219 - accuracy: 0.9937
Epoch 4/50
156/156 [==============================] - 2s 12ms/step - loss: 0.2282 - accuracy: 0.9457
Epoch 5/50
156/156 [==============================] - 2s 14ms/step - loss: 0.0258 - accuracy: 0.9924
Epoch 6/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0154 - accuracy: 0.9961
Epoch 7/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0129 - accuracy: 0.9969
Epoch 8/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0114 - accuracy: 0.9973
Epoch 9/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0103 - accuracy: 0.9975
Epoch 10/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0096 - accura

In [17]:
model2.fit(trainX,trainYB,epochs=50,batch_size=10000)

2022-10-23 08:01:07.067562: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 673271568 exceeds 10% of free system memory.


Epoch 1/50
156/156 [==============================] - 3s 14ms/step - loss: 0.2779 - accuracy: 0.9272
Epoch 2/50
156/156 [==============================] - 2s 13ms/step - loss: 0.2373 - accuracy: 0.9279
Epoch 3/50
156/156 [==============================] - 2s 13ms/step - loss: 0.2302 - accuracy: 0.9287
Epoch 4/50
156/156 [==============================] - 2s 13ms/step - loss: 0.2169 - accuracy: 0.9307
Epoch 5/50
156/156 [==============================] - 2s 13ms/step - loss: 0.1946 - accuracy: 0.9348
Epoch 6/50
156/156 [==============================] - 2s 13ms/step - loss: 0.1694 - accuracy: 0.9412
Epoch 7/50
156/156 [==============================] - 2s 13ms/step - loss: 0.1467 - accuracy: 0.9484
Epoch 8/50
156/156 [==============================] - 2s 13ms/step - loss: 0.1268 - accuracy: 0.9553
Epoch 9/50
156/156 [==============================] - 2s 13ms/step - loss: 0.1112 - accuracy: 0.9609
Epoch 10/50
156/156 [==============================] - 2s 13ms/step - loss: 0.0951 - accura

In [19]:
model1.save('modelA.h5')
model2.save('modelB.h5')